In [1]:
import nltk
nltk.download("punkt")
nltk.download("wordnet")
nltk.download("semcor") # downloads the .zip file, but doesn't unzip it
nltk.download("stopwords")
nltk.download("averaged_perceptron_tagger")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package semcor to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [1]:
! unzip -q /root/nltk_data/corpora/semcor.zip -d /root/nltk_data/corpora # after this, data in /root/nltk_data/corpora/semcor

In [3]:
from gensim.models import KeyedVectors
import gensim.downloader as api

# Download the pre-trained Word2Vec model
W2V = api.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [4]:
! pip install num2words

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.3/143.3 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=f9ddb25bc8ea72b32ced38b69ee6c1a81307c451cc8da9b6edb1f2fec9e949f2
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [5]:
import nltk
from nltk import word_tokenize
from nltk.corpus import semcor # corpus reader: https://github.com/nltk/nltk/blob/develop/nltk/corpus/reader/semcor.py
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer

import random
import numpy as np
from tqdm.notebook import tqdm
from string import punctuation
from num2words import num2words
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Custom stopwords
EXTRA_SW = [
    "''",
    "'s",
    "``"
]

SW = stopwords.words("english")
SW += [p for p in punctuation]
SW += EXTRA_SW

lemmatizer = WordNetLemmatizer()

def cosineSimilarity(a, b):
    cs = np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))
    return cs
def isNumber(s):
    try:
        float(s)
        return True
    except ValueError:
        return False
def n2w(w):
    # converts given n to word form if n is numeric
    if isNumber(w) and w.lower() != "infinity" and w.lower() != "nan":
        w = num2words(w)
    return w
def lemmatize(w, tag):
    if tag is None:
        return lemmatizer.lemmatize(w)
    else:
        return lemmatizer.lemmatize(w, tag)

def clean(tokens):
    tagged = nltk.pos_tag(tokens)
    lemmatized = [lemmatize(w, treebank2wn(tag)) for w, tag in tagged]
    cleaned = [n2w(w) for w in lemmatized if w.lower() not in SW]
    return cleaned
def getVec(w):
    # Returns (300,) shaped numpy array
    try:
        v = W2V[w]
        return v
    except KeyError:
        return None # ignore words not in vocab

def syn2sense(syn):
    # get the sense (= lemma.postag.num) for a given synset
    s = syn.name()
    # n = ".".join(s.split(".")[-2:]) # n.01 and v.01 are different senses (eg: ash.n.01, ash.v.01)
    return s
def treebank2wn(ttag):
    if ttag.startswith("J"):
        return wn.ADJ
    elif ttag.startswith("V"):
        return wn.VERB
    elif ttag.startswith("N"):
        return wn.NOUN
    elif ttag.startswith("R"):
        return wn.ADV
    else:
        return None

def sent2vec(tokens):

    v = 0
    n = 0

    for w in tokens:

        # Check if w is a named entity (TODO: use wordnet NE tag directly instead of below approach)
        tkns = word_tokenize(w)

        if len(tkns) > 1:
            for t in tkns:
                vt = getVec(t)
                if vt is not None:
                    n += 1
                    v += vt
        else:
            vw = getVec(w)
            if vw is not None:
                n += 1
                v += vw

    if n == 0: # when tokens is empty or no token in word2vec
        v = None
    else:
        v /= n

    return v

def parse(d):
    # d (nltk.corpus.reader.semcor.SemcorSentence) : can have lists as elements or nltk.tree.Tree

    tokens = []
    senses = []

    for e in d:

        if isinstance(e, nltk.tree.Tree):

            # e.label() returns a nltk.corpus.reader.wordnet.Lemma object or simply a string (of the form word.pos.num)
            lemma = e.label()

            if isinstance(lemma, nltk.corpus.reader.wordnet.Lemma):
                synset = lemma.synset() # nltk.corpus.reader.wordnet.Synset
                sense = syn2sense(synset)
            else:
                sense = None # ignore all tagged senses which aren't in WN (i.e. present as Lemma)

            le = len(e)
            if le == 1:
                w = e[0]
                if isinstance(w, nltk.tree.Tree) or isinstance(w, list):
                    # ignore w.label()
                    lw = len(w)
                    w = " ".join([w[i] for i in range(lw)])
            else:
                w = " ".join([e[i] for i in range(le)])

        elif isinstance(e, list):
            w = e[0]
            sense = None

        else:
            invtype = type(e)
            raise TypeError(f"Invalid type: {invtype}")

        if w:
            tokens.append(w)
            senses.append(sense)

    return tokens, senses

def getCandidates(w, tag):
    # Get candidate sense vectors and labels of a word w

    w = w.replace(".", "") # "Sept." becomes "Sept"
    w = w.replace("-", "") # re-elected becomes "relected"

    # Handle ngrams (like "united states")
    tkns = word_tokenize(w)
    if len(tkns) > 1:
        tagged = nltk.pos_tag(tkns)
        tags = [treebank2wn(p[1]) for p in tagged]
        ltkns = [lemmatize(w, t) for w, t in zip(tkns, tags)]
        w = "_".join(ltkns)

    syns = wn.synsets(w, tag)

    if len(syns) == 0:
        w = "_".join(tkns) # cases where lemmatization doesn't help ("agreed upon")
        syns = wn.synsets(w, tag)

    sense_vectors = []
    sense_labels = []

    for syn in syns:

        label = syn2sense(syn)

        defn = syn.definition() # TODO: Implement the extended Lesk algorithm that uses related synsets as well

        defn = defn.replace("_", " ")
        defn = defn.replace("-", " ")

        tkns = word_tokenize(defn)
        if len(tkns) == 0:
            raise ValueError(f"0 tokens found: {defn}")

        clnd = clean(tkns)
        if len(clnd) < 2:
            clnd = tkns # don't remove stopwords if the sentence is almost entirely made up of them

        sv = sent2vec(clnd)

        if sv is None:
            print(f"Empty sense vector. Word: {w}, Definition: {defn}, Cleaned: {clnd}. Using a random vector as sense.")
            sv = np.random.rand(300,)

        sense_vectors.append(sv)
        sense_labels.append(label)

    return sense_vectors, sense_labels # returns empty lists if no synsets found

data = semcor.tagged_sents(tag = "sem") # 37176 sentences, 224716 tagged words, 34189 unique senses

n_total = 0
n_correct = 0
n_samples = 0

true = []
pred = []

for d in data:

    try:

        tokens, senses = parse(d)
        n_tokens = len(tokens)

        # Tag and lemmatize tokens, don't remove stopwords here
        tagged = nltk.pos_tag(tokens)
        tags = [treebank2wn(p[1]) for p in tagged]
        tokens = [lemmatize(w, tag) for w, tag in zip(tokens, tags)]

        for i in range(n_tokens):

            w = tokens[i]
            tag = tags[i]
            s_true = senses[i]

            if not isinstance(w, str):
                raise TypeError(f"Invalid type: {type(w)} : {w} : {tokens}")

            # Don't predict for words that aren't sense-tagged
            if s_true is None:
                continue

            # Get context for w (all words in the sentence except w)
            context = tokens.copy()
            del context[i] # more efficient than .pop(i)

            # Remove stopwords and punctuation from context to reduce #elements in the context
            # These don't contribute much to the semantic overlap anyways
            cleaned = clean(context)
            if len(cleaned) < 2:
                cleaned = context # if almost all words are stopwords, don't remove any

            # Get context vector by average w2v vectors for each word
            cv = sent2vec(cleaned)

            if cv is None:
                print(f"Empty context vector. Word: {w}, Cleaned: {cleaned}, Tokens: {tokens}. Using a random vector as context.")
                cv = np.random.rand(300,)

            # Get WordNet candidate senses
            sense_vectors, sense_labels = getCandidates(w, tag)
            n_candidates = len(sense_labels)

            s_pred = None
            if n_candidates == 0:
                # Try without pos tag
                sense_vectors, sense_labels = getCandidates(w, None)
                n_candidates = len(sense_labels)
                if n_candidates == 0:
                    # print(f"No synsets found. Word: {w}, Sense: {s_true}") # don't print, too many NE's in the data
                    s_pred = random.choice(["group.n.01", "person.n.01", "location.n.01"]) # most likely an NE

            # Use cosine similarity to get the best senses
            best = -1
            for j in range(n_candidates):
                sv = sense_vectors[j]
                cs = cosineSimilarity(cv, sv)
                if cs > best:
                    best = cs
                    s_pred = sense_labels[j]

            if s_true == s_pred:
                n_correct += 1
            n_total += 1

            true.append(s_true)
            pred.append(s_pred)

    except Exception as e:
        print(f"Error at: {n_samples}")
        print(str(e))
        raise ValueError("Error")

    n_samples += 1

    if n_samples%200 == 0:
        print(f"{n_samples} sentences processed")
        acc = (n_correct/n_total)*100
        print(f"Accuracy: {acc:.4f}")
        print()

pred_sense_set = set(pred)
true_sense_set = set(true)
all_senses = sorted(list(true_sense_set.union(pred_sense_set)))
not_predicted = true_sense_set - pred_sense_set
extra_predicted = pred_sense_set - true_sense_set

acc = accuracy_score(true, pred)
prec = precision_score(true, pred, average = "macro")
rec = recall_score(true, pred, average = "macro")
f1 = f1_score(true, pred, average = "macro")

print(f"Accuracy: {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall: {rec:.4f}")
print(f"F1-Score: {f1:.4f}")

def predict(sent):

    senses = []
    tokens = word_tokenize(sent)
    # Tag and lemmatize tokens, don't remove stopwords here
    tagged = nltk.pos_tag(tokens)
    tags = [treebank2wn(p[1]) for p in tagged]
    tokens = [lemmatize(w, tag) for w, tag in zip(tokens, tags)]
    n_tokens = len(tokens)

    for i in range(n_tokens):

        w = tokens[i]
        tag = tags[i]

        # Get context for w (all words in the sentence except w)
        context = tokens.copy()
        del context[i] # more efficient than .pop(i)

        # Get context vector by average w2v vectors for each word
        cv = sent2vec(context)

        if cv is None:
            print(f"Empty context vector. Word: {w}, Tokens: {tokens}. Using a random vector as context.")
            cv = np.random.rand(300,)

        # Get WordNet candidate senses
        sense_vectors, sense_labels = getCandidates(w, tag)
        n_candidates = len(sense_labels)

        s_pred = None
        if n_candidates == 0:
            # Try without pos tag
            sense_vectors, sense_labels = getCandidates(w, None)
            n_candidates = len(sense_labels)
            if n_candidates == 0:
                print(f"No synsets found: {w}")
                s_pred = None

        # Use cosine similarity to get the best senses
        best = -1
        for j in range(n_candidates):
            sv = sense_vectors[j]
            cs = cosineSimilarity(cv, sv)
            if cs > best:
                best = cs
                s_pred = sense_labels[j]

        senses.append(s_pred)

    return senses

sents = [
    "On combustion of coal we get ash",
    "The bank is located in the city near the river",
]

for sent in sents:
    senses = predict(sent)
    for s in senses:
        if s is not None:
            print(s, ":", wn.synset(s).definition())
    print()

200 sentences processed
Accuracy: 44.0917

400 sentences processed
Accuracy: 41.7053

600 sentences processed
Accuracy: 41.0985

800 sentences processed
Accuracy: 40.2001

1000 sentences processed
Accuracy: 40.4275

1200 sentences processed
Accuracy: 40.2418

1400 sentences processed
Accuracy: 40.3064

1600 sentences processed
Accuracy: 40.7091

1800 sentences processed
Accuracy: 40.7059

2000 sentences processed
Accuracy: 40.6643

2200 sentences processed
Accuracy: 40.6321

Empty context vector. Word: Cancer, Cleaned: ['``', "''", '!'], Tokens: ['``', 'Cancer', "''", '!']. Using a random vector as context.
Empty context vector. Word: By no means, Cleaned: ['.'], Tokens: ['By no means', '.']. Using a random vector as context.
Empty context vector. Word: For instance, Cleaned: [':'], Tokens: ['For instance', ':']. Using a random vector as context.
Empty context vector. Word: Death, Cleaned: ['!'], Tokens: ['Death', '!']. Using a random vector as context.
2400 sentences processed
Accurac

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.3787
Precision: 0.4133
Recall: 0.4007
F1-Score: 0.3774
No synsets found: of
No synsets found: we
on.r.03 : in a state required for something to function or be effective
combustion.n.01 : a process in which a substance reacts with oxygen to give heat and light
ember.n.01 : a hot fragment of wood or coal that is left from a fire and is glowing or smoldering
get.v.01 : come into the possession of something concrete or abstract
ash.n.01 : the residue that remains when something is burned

No synsets found: The
No synsets found: the
No synsets found: the
bank.n.07 : a slope in the turn of a road or track; the outside is higher than the inside in order to reduce the effects of centrifugal force
be.v.03 : occupy a certain position or area; be somewhere
situate.v.01 : determine or indicate the place, site, or limits of, as if by an instrument or by a survey
in.r.01 : to or toward the inside of
city.n.01 : a large and densely populated urban area; may include several independent adm